In [ ]:
DRIVE_PREFIX = '/content/drive/MyDrive/Colab Notebooks/Code LLMs/'
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install accelerate -U
#!pip install transformers[torch]

In [ ]:
from huggingface_hub import notebook_login


In [ ]:
notebook_login()


In [ ]:
import sys
sys.path.append(DRIVE_PREFIX)
import data_loader

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class BERTLikeModel(nn.Module):
  # default relatively small
    def __init__(self,
                 vocab_size,
                 hidden_size=768,
                 num_layers=12,
                 num_attention_heads=12,
                 max_position_embeddings=512):

        super(BERTLikeModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=num_attention_heads,
                dim_feedforward=4 * hidden_size,
            ) for _ in range(num_layers)
        ])

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=num_attention_heads,
                dim_feedforward=4 * hidden_size,
            ),
            num_layers=num_layers,
        )

        self.mlm_head = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_ids, attention_mask):
        # Embedding
        embedded = self.embedding(input_ids)

        # Transformer encoding
        transformer_output = self.transformer_encoder(embedded, src_key_padding_mask=~attention_mask)

        # Masked Language Modeling head
        mlm_logits = self.mlm_head(transformer_output)

        return mlm_logits


In [ ]:
train, test = data_loader.load_dataset(drive_prefix=DRIVE_PREFIX)

In [ ]:
# make a tokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
import os

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])


In [ ]:
tokenizer.pre_tokenizer = Whitespace()
path_to_codenet = DRIVE_PREFIX + "python_codenet/"
files = [(path_to_codenet + file) for file in os.listdir(path_to_codenet) ]
tokenizer.train(files, trainer)

KeyboardInterrupt: ignored

In [ ]:
tokenizer.save(DRIVE_PREFIX + "py_tokenizer.json")

In [ ]:
# load the tokenizer
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file= DRIVE_PREFIX + "py_tokenizer.json")


In [ ]:
#DEBUG pretend we already have a tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Microsoft/codebert-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples['contents'], truncation=True, max_length=max_length)
    #return tokenizer.encode([examples['contents']])#, truncation=True, max_length=max_length)

In [ ]:

def group_texts(examples, block_size=128):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # drop the remainder
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset

In [ ]:
train, test = data_loader.load_dataset(drive_prefix=DRIVE_PREFIX)

train = train.dropna(subset=['contents'], how='any', inplace=False)
test = test.dropna(subset=['contents'], how='any', inplace=False)



In [ ]:
# for idx, item in train.iterrows():
#   if type(item['contents']) != str:
#     print("row which is not a string")


In [ ]:
test = Dataset.from_pandas(test)


In [ ]:
# pre-train models of various sizes

random_state=5535
num_epochs = 3
learning_rate = 2e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenized_eval = test.map(
      preprocess_function,
      batched=True,
      num_proc=4,
      remove_columns = test.column_names
  )

modeling_eval = tokenized_eval.map(group_texts,
                                   batched=True,
                                   num_proc=4)



Map (num_proc=4):   0%|          | 0/5333 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5333 [00:00<?, ? examples/s]

In [ ]:
modeling_eval

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 12318
})

In [ ]:
from transformers import AutoModelForMaskedLM



In [ ]:
for fraction in [0.25, 0.5, 0.75, 1]:
  #model = BERTLikeModel(vocab_size = len(tokenizer))
  model = AutoModelForMaskedLM.from_pretrained("roberta-base")
  model.to(device)

  train_frac = train.sample(frac=fraction, random_state=random_state)

  train_frac = Dataset.from_pandas(train_frac)

  optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss(ignore_index=-100)

  # pre-proc our datasets
  tokenized_frac = train_frac.map(
      preprocess_function,
      batched=True,
      num_proc=4,
      remove_columns=train_frac.column_names
  )

  modeling_train = tokenized_frac.map(group_texts,
                                   batched=True,
                                   num_proc=4)

  training_args = TrainingArguments(
    output_dir=f"{str(fraction).replace('.','')}_python",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=modeling_train,
      eval_dataset=modeling_eval,
      data_collator=data_collator,
  )

  trainer.train()

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/5332 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5332 [00:00<?, ? examples/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Map (num_proc=4):   0%|          | 0/15998 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.682500,0.598901
2,0.576100,0.510370
3,0.525700,0.470544


Map (num_proc=4):   0%|          | 0/21330 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21330 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.665400,0.562145
2,0.544100,0.480400
3,0.506200,0.445161
